In [10]:
import pandas as pd
import numpy as np
import sqlite3
import time

data_dir = 'R6/'
engine = sqlite3.connect('yahoo')

In [ ]:
# Unbiased Offline Evaluation of Contextual-bandit-based News Article Recommendation Algorithms 
# https://arxiv.org/pdf/1003.5956.pdf
# Dataset: https://webscope.sandbox.yahoo.com/catalog.php?datatype=r&did=49

t0 = time.time()

articles_df = pd.DataFrame(columns=['2', '3', '4', '5', '6', '1'])

ITEM_LIMIT = sys.maxsize
num_iters = 0

with open(data_dir + 'clicks_1.txt') as f:
    for line in f:
        num_iters += 1
        if num_iters > ITEM_LIMIT:
            break
        parts = line.strip().split('|')
        unix_timestamp, disp_article_id, clicked = parts[0].split()
#         print('Unix Timestamp: {0}, Displayed Article Id: {1}, Clicked: {2}'.format(unix_timestamp, disp_article_id, clicked))
        for i in range(2, len(parts)):
            # Extract article vector information.
            article_info = parts[i].split()
            article_id = article_info[0]
            if article_id in articles_df.index:
                continue
            article_info_parts = list(map(lambda x : x.split(':')[1], article_info[1:]))
            article_info = dict(zip(articles_df.columns, article_info_parts))
            articles_df.loc[article_id] = pd.Series(article_info)
#         break 

t1 = time.time()
print('Finished processing {0} items in {1} seconds.'.format(num_iters-1, t1 - t0))
        
print(articles_df.shape)
articles_df

In [15]:
articles_df.to_sql('articles', engine, if_exists='replace')